# KAN + NTK + MNIST
Purpose: Fit a NTK to a KAN intended for the MNIST dataset.

This is the Accuracy variant, which means it seeks to find the highest performance using as much of the dataset as possible, while balancing the parameter count.

Furthermore, the PyTorch Lightning library is used for convenience.

In [1]:
# Imports
import numpy as np
import pandas as pd
import torch
from torch import nn
import lightning as L
import torch.nn.functional as F
from torchvision.datasets import MNIST
from torch.utils.data import Dataset, DataLoader, Subset, random_split
from torchvision import transforms
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
import matplotlib.pyplot as plt

import sys
sys.path.append('convkans/kan_convolutional')
from KANLinear import *

# Setup Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Setup Randomness -- https://lightning.ai/docs/pytorch/stable/common/trainer.html
L.seed_everything(68812, workers=True) # <--- Adjust the seed here for each trial run for the 5 seed values [42, 998244353, 10122024, 1000000007, 68812].

# CUDA Efficiency
if torch.cuda.is_available():
    torch.set_float32_matmul_precision('high')
    print("Configured CUDA Precision")

Seed set to 68812


Configured CUDA Precision


In [2]:
# Dataset Setup -- Inspired by Hugo's Dataset Reformatting
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = MNIST("./Experiments/temp/", train=True, download=True, transform=transform)
test_dataset = MNIST("./Experiments/temp/", train=False, download=True, transform=transform)

train_size = int(len(train_dataset) * 0.99)
val_size = int(len(train_dataset)) - train_size # Quite low validation size, but this is done to allow for greater model accuracy.
train_set, val_set = random_split(train_dataset, [train_size, val_size])

# Reformatted, due to odd issues when using NTK on it
class LCDataset(Dataset): # Lightning Compatible Dataset
    def __init__(self, dataset, num_classes, limit=-1):
        self.limit = limit
        self.num_classes = num_classes
        if self.limit != -1:
            sub = list(np.random.permutation(np.arange(len(dataset)))[0:self.limit]) # Take a random sample of the first some elements.
            self.dataset = Subset(dataset, sub)
        else:
            self.dataset = dataset
    
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        x, y = self.dataset[idx]
        y_one_hot = torch.zeros(self.num_classes)
        y_one_hot[y] = 1
        return x, y_one_hot

batch_size = 64
ntk_loader = DataLoader(LCDataset(train_dataset, num_classes=10, limit=800), batch_size=batch_size, num_workers=10)
train_loader = DataLoader(LCDataset(train_set, num_classes=10), batch_size=batch_size, shuffle=True, num_workers=10)
val_loader = DataLoader(LCDataset(val_set, num_classes=10), batch_size=batch_size, shuffle=False, num_workers=10)
test_loader = DataLoader(LCDataset(test_dataset, num_classes=10), batch_size=batch_size, num_workers=10)

In [3]:
# Model Declaration
class ClassicKAN(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.BatchNorm2d(1),
            nn.MaxPool2d(2),
            nn.Flatten(1),
            KAN(layers_hidden=[14*14, 64, 10], grid_size=2, spline_order=2)
        )
    
    def forward(self, x):
        return self.net(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = F.cross_entropy(y_pred, y)
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        v1 = torch.argmax(y_pred, dim=1)
        v2 = torch.argmax(y, dim=1)
        loss = F.cross_entropy(y_pred, y)
        accuracy = torch.sum(torch.eq(v1, v2)) / len(y)
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", accuracy, prog_bar=True) # Observe model accuracy with time

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        v1 = torch.argmax(y_pred, dim=1)
        v2 = torch.argmax(y, dim=1)
        accuracy = torch.sum(torch.eq(v1, v2)) / len(y)
        self.log("accuracy", accuracy) # Accuracy for Accuracy vs. Parameter Experiment

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        return self(batch)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=2e-4, weight_decay=2e-3)
        return optimizer

In [4]:
# Train + Test + Results
model = ClassicKAN()
trained_model = L.Trainer(max_epochs=0, deterministic=True, logger=CSVLogger(save_dir="Experiments/logs", name="MNISTNTKKANAcc", version="fin"), callbacks=[EarlyStopping(monitor="val_loss", mode="min", patience=10)])
trained_model.fit(model, train_loader, val_loader)
trained_model.test(model, test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/martin/Documents/DataScienceEnv/GenInst/lib/python3.12/site-packages/lightning/fabric/loggers/csv_logs.py:268: Experiment logs directory Experiments/logs/MNISTNTKKANAcc/fin exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params | Mode 
--------------------------------------------
0 | net  | Sequential | 79.1 K | train
--------------------------------------------
79.1 K    Trainable params
0         Non-trainable params
79.1 K    Total params
0.316     Total estimated model params size (MB)
10        Modules in train mode
0         Modules in eval mode


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=0` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |                                                | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │    0.0778999999165535     │
└───────────────────────────┴───────────────────────────┘

[{'accuracy': 0.0778999999165535}]

In [5]:
# Apply NTK
%run -i './Experiments/introduction_code_modified.py'

def cross_entropy_loss_batch(y_hat, y):
    return F.cross_entropy(y_hat, y, reduction='none')

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
ntk_model = GaussianFit(model=model, device=device, noise_var=0.0)
ntk_model.fit(ntk_loader, optimizer, MSELoss_batch) # MSELoss seems to work better for NTK Models

In [6]:
def check_ntk_acc(model, dataloader):
    res = 0.0
    sumlength = 0
    model.eval()
    model.to(device)
    for it in iter(dataloader):
        x, y = it
        x = x.to(device)
        y = y.to(device)
        sumlength += len(x)
        res += (torch.argmax(model.forward(x), dim=1) == torch.argmax(y, dim=1)).sum()
    model.train()
    return res / sumlength

In [7]:
print(f'NTK Accuracy: {check_ntk_acc(ntk_model, test_loader)}')

NTK Accuracy: 0.7701999545097351


In [8]:
# R: True Value, C: Predicted Value
def make_predict_matrix(model, dataloader):
    res = np.zeros(shape=(10, 10), dtype=int)
    model = model.to(device)
    for it in iter(dataloader):
        x, y = it
        x = x.to(device)
        y = y.to(device)
        x_arg = torch.argmax(model.forward(x), dim=1)
        y_arg = torch.argmax(y, dim=1)
        for i in range(len(x_arg)):
            res[y_arg[i], x_arg[i]] += 1
    return res

In [9]:
print(make_predict_matrix(ntk_model, test_loader))

[[ 838   14   11   22    9   15   17    9   26   19]
 [   0 1119    2    3    5    0    3    1    2    0]
 [   9   32  722   45   20    3   15   50  124   12]
 [   3    8   11  861    4   29   12   19   33   30]
 [   3   16   10    5  778    6   12   17   11  124]
 [   3   22    3   73   23  644   22   17   50   35]
 [   8   10    8    8   20   11  850   12   18   13]
 [   7   33   12   10   15    1    1  867    9   73]
 [   9   13    4   35   21   19    6    8  801   58]
 [   8   14    8   16   42    7    1   20   11  882]]
